In [1]:
# Author:Ning Pei Ding
# Date: 02/05/2020
import pandas as pd
import numpy as np
import scipy.io
import statsmodels.api as sm
# If you can not load stasmodels package, please input 'pip install statsmodel' in your terminal.

In [17]:
#Load data
mat = scipy.io.loadmat('china.mat')

In [5]:
#Explore the data
for i in mat.keys():
    print(i, len(mat[i]))

__header__ 76
__version__ 3
__globals__ 0
china_daily_dollarvol 3845
china_daily_mv 3845
china_daily_pi 3845
china_daily_ri 3845
china_daily_up 3845
None 1
china_dwcx 3845
china_dweb 3845
china_dwed 3845
china_dwee 3845
china_dwen 3845
china_dwev 3845
china_dwfc 3845
china_dwic 3845
china_dwnd 3845
china_dwnm 3845
china_dwnp 3845
china_dwre 3845
china_dwse 3845
china_dwsl 3845
china_dwta 3845
china_icbic 3845
china_monthly_mv 3845
china_monthly_pi 3845
china_monthly_ri 3845
china_monthly_up 3845
china_tr3 3845
ddates 8349
mdates 385
__function_workspace__ 1


In [11]:
#Data Process
mret = np.array(mat['china_monthly_ri'])
market = np.array(mat['china_monthly_mv'])
rankrt = np.stack([(mret[:,i] - mret[:,i-1])/mret[:,i-1] for i in range(1,mret.shape[1])], axis = -1)
mk = np.stack([market[:,i] for i in range(1,mret.shape[1])], axis = -1)
momentum = np.stack([np.prod(rankrt[:,i-11:i] + 1,axis = 1) -1 for i in range(11,rankrt.shape[1])], axis = -1)
momentum= np.concatenate((np.zeros([momentum.shape[0],11]), momentum), axis=1)

In [2]:
market

NameError: name 'market' is not defined

In [13]:
nc = rankrt.shape[1]
nr = rankrt.shape[0]

In [2]:
#Chunk the data by the percentage
def percentage(per, data):
    res = []
    for i in per:
        res.append(np.percentile(data,i))
    return res        

In [27]:
#Calculate the mean return and weighted return and OLS regersssion result
def calresult(rankrt, mk, order, percent, start, end, size = 'Full'):
    mtr_mean = []
    vr_mean = []
    ewret = []
    vmret = []
    percent = percent
    for i in range(start, end,1):
        rankrt_curr1 = rankrt[(~np.isnan(rankrt[:,i]))&(~np.isnan(rankrt[:,i-1]))&(~np.isnan(mk[:,i-1]))&(~np.isnan(order[:,i-1])),i-1]
        rankmk1 = mk[(~np.isnan(rankrt[:,i]))&(~np.isnan(rankrt[:,i-1]))&(~np.isnan(mk[:,i-1]))&(~np.isnan(order[:,i-1])),i-1]
        rankrt_next1 = rankrt[(~np.isnan(rankrt[:,i]))&(~np.isnan(rankrt[:,i-1]))&(~np.isnan(mk[:,i-1]))&(~np.isnan(order[:,i-1])),i]
        rankorder1 =  order[(~np.isnan(rankrt[:,i]))&(~np.isnan(rankrt[:,i-1]))&(~np.isnan(mk[:,i-1]))&(~np.isnan(order[:,i-1])),i-1]
        if len(rankrt_curr1) == 0 | len(rankorder1) == 0:
            continue
        if size == 'Full':
            rankrt_curr = rankrt_curr1
            rankmk = rankmk1
            rankrt_next = rankrt_next1
            rankorder = rankorder1
        if size == 'Big':
            rankrt_curr = rankrt_curr1[rankmk1 >= np.nanmedian(rankmk1)]
            rankmk = rankmk1[rankmk1 >= np.nanmedian(rankmk1)]
            rankrt_next = rankrt_next1[rankmk1 >= np.nanmedian(rankmk1)]
            rankorder = rankorder1[rankmk1 >= np.nanmedian(rankmk1)]
        if size == 'Small':
            rankrt_curr = rankrt_curr1[rankmk1 < np.nanmedian(rankmk1)]
            rankmk = rankmk1[rankmk1 < np.nanmedian(rankmk1)]
            rankrt_next = rankrt_next1[rankmk1 < np.nanmedian(rankmk1)]
            rankorder = rankorder1[rankmk1 < np.nanmedian(rankmk1)]
        bound = percentage(percent, rankorder) 
        mmean = []
        vr=[]
        mmean.append(np.mean(rankrt_next[rankorder <= bound[0]]))
        vr.append(np.dot(rankrt_next[rankorder <= bound[0]].T , (rankmk[rankorder <= bound[0]]/np.nansum(rankmk))))
        ewret.append(np.mean(rankrt_next))
        vmret.append(np.dot(rankrt_next.T, rankmk/np.nansum(rankmk)))
        if len(bound) > 1:
            for j in range(1,len(bound)):
                mmean.append(np.mean(rankrt_next[(rankorder > bound[j-1]) & (rankorder < bound[j])]))
                vr.append(np.dot(rankrt_next[(rankorder > bound[j-1]) & (rankorder < bound[j])].T, (rankmk[(rankorder > bound[j-1]) & (rankorder < bound[j])]/np.nansum(rankmk))))
            mmean.append(np.mean(rankrt_next[rankorder > bound[-1]]))
            vr.append(np.dot(rankrt_next[rankorder > bound[-1]].T, (rankmk[rankorder > bound[-1]]/np.nansum(rankmk))))
        mtr_mean.append(mmean)
        vr_mean.append(vr)
    mtr_mean = np.array(mtr_mean)
    vr_mean = np.array(vr_mean)
    diff1 = mtr_mean[:,-1] - mtr_mean[:,0]
    diff2 = vr_mean[:,-1] - vr_mean[:,0]
#     ewresults = sm.OLS(diff1, sm.add_constant(ewret)).fit()
#     vmresults = sm.OLS(diff2, sm.add_constant(vmret)).fit()
    print('Mean return in five part: \n\n', np.nanmean(mtr_mean, axis = 0)*100)
    print('----------------------------------------------------------------------n')
    print('Weighted mean return in five part: \n\n', np.nanmean(vr_mean, axis = 0)*100)
    print('----------------------------------------------------------------------n')
    print('Difference between high part and low part: \n\n', 100*np.nanmean(diff1, axis = 0),100*np.nanmean(diff2, axis= 0))
    print('----------------------------------------------------------------------n')
#     print('Alpha of mean return is \n\n', ewresults.params[0])
#     print('----------------------------------------------------------------------n')
#     print('Stat result of regression(mean): \n')
#     print('----------------------------------------------------------------------n')
#     print(ewresults.summary())
#     print('----------------------------------------------------------------------n')
#     print('Alpha of weighted mean return is \n\n', vmresults.params[0])
#     print('----------------------------------------------------------------------n')
#     print('Stat result of regression(weighted mean): \n')
#     print('----------------------------------------------------------------------n')
#     print(vmresults.summary())

# Short-Term Revesal

## 1.Full Size

In [31]:
%%time
mat = scipy.io.loadmat('china.mat')
mret = np.array(mat['china_monthly_ri'])
market = np.array(mat['china_monthly_mv'])
rankrt = np.stack([(mret[:,i] - mret[:,i-1])/mret[:,i-1] for i in range(1,mret.shape[1])], axis = -1)
mk = np.stack([market[:,i] for i in range(1,mret.shape[1])], axis = -1)
momentum = np.stack([np.prod(rankrt[:,i-11:i] + 1,axis = 1) -1 for i in range(11,rankrt.shape[1])], axis = -1)
momentum= np.concatenate((np.zeros([momentum.shape[0],11]), momentum), axis=1)
nc = rankrt.shape[1]
nr = rankrt.shape[0]
calresult(rankrt,mk,momentum, [20, 40, 60, 80], 87, rankrt.shape[1])


Mean return in five part: 

 [1.72168821 1.74031106 1.92944251 1.67570641 1.32186894]
----------------------------------------------------------------------n
Weighted mean return in five part: 

 [0.20680867 0.19716919 0.18327156 0.24460084 0.31865894]
----------------------------------------------------------------------n
Difference between high part and low part: 

 -0.3998192664540337 0.11185027385453507
----------------------------------------------------------------------n
CPU times: user 4.82 s, sys: 889 ms, total: 5.71 s
Wall time: 5.96 s


## 2. Small Size

In [18]:
calresult(rankrt,mk,rankrt, [20, 40, 60, 80], 87, rankrt.shape[1], 'Small')

Mean return in five part: 

 [2.65644583 2.67902578 2.27034644 1.95421601 1.18643281]
----------------------------------------------------------------------n
Weighted mean return in five part: 

 [0.46591333 0.48796539 0.40418741 0.35426638 0.21179788]
----------------------------------------------------------------------n
Difference between high part and low part: 

 -1.4700130133646323 -0.2541154447211612
----------------------------------------------------------------------n
Alpha of mean return is 

 -0.013706729330908062
----------------------------------------------------------------------n
Stat result of regression(mean): 

----------------------------------------------------------------------n
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:

## 3. Big Size

In [20]:
calresult(rankrt,mk,rankrt, [20, 40, 60, 80], 87, rankrt.shape[1], 'Big')

Mean return in five part: 

 [1.36609174 1.35837662 1.25099663 1.00732798 0.76846144]
----------------------------------------------------------------------n
Weighted mean return in five part: 

 [0.08896889 0.18589939 0.23349747 0.25271378 0.19144175]
----------------------------------------------------------------------n
Difference between high part and low part: 

 -0.5976303041239813 0.10247286287679268
----------------------------------------------------------------------n
Alpha of mean return is 

 -0.005713219295516926
----------------------------------------------------------------------n
Stat result of regression(mean): 

----------------------------------------------------------------------n
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:

# Market Size Effect

## 1. Full Size 

In [23]:
calresult(rankrt,mk,mk, [20, 40, 60, 80], 87, rankrt.shape[1])

Mean return in five part: 

 [2.87543207 1.76453689 1.42551736 1.19126092 0.99202834]
----------------------------------------------------------------------n
Weighted mean return in five part: 

 [0.10104943 0.09798825 0.11791932 0.15675144 0.60191187]
----------------------------------------------------------------------n
Difference between high part and low part: 

 -1.8834037347317207 0.500862438958551
----------------------------------------------------------------------n
Alpha of mean return is 

 -0.015323089086231522
----------------------------------------------------------------------n
Stat result of regression(mean): 

----------------------------------------------------------------------n
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.124
Model:                            OLS   Adj. R-squared:                  0.121
Method:                 Least Squares   F-statistic:  

## 2. Small Size 

In [25]:
calresult(rankrt,mk,mk, [20, 40, 60, 80], 87, rankrt.shape[1], 'Small')

Mean return in five part: 

 [3.37822957 2.36256835 1.90467764 1.62515759 1.45397739]
----------------------------------------------------------------------n
Weighted mean return in five part: 

 [0.36047946 0.36766012 0.37148161 0.38978342 0.43137012]
----------------------------------------------------------------------n
Difference between high part and low part: 

 -1.9242521784551228 0.07089066248142267
----------------------------------------------------------------------n
Alpha of mean return is 

 -0.017938150779960523
----------------------------------------------------------------------n
Stat result of regression(mean): 

----------------------------------------------------------------------n
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:

## 3. Big Size

In [27]:
calresult(rankrt,mk,mk, [20, 40, 60, 80], 87, rankrt.shape[1], 'Big')

Mean return in five part: 

 [1.38364433 1.183013   1.20435837 0.99302371 0.99059228]
----------------------------------------------------------------------n
Weighted mean return in five part: 

 [0.07446351 0.07822272 0.10344359 0.13125659 0.56648403]
----------------------------------------------------------------------n
Difference between high part and low part: 

 -0.39305204684699097 0.4920205208750943
----------------------------------------------------------------------n
Alpha of mean return is 

 -0.0021579135499525718
----------------------------------------------------------------------n
Stat result of regression(mean): 

----------------------------------------------------------------------n
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.087
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic

# Momentum

## 1. Full Size

In [30]:
calresult(rankrt,mk,momentum, [20, 40, 60, 80], 87, rankrt.shape[1])

Mean return in five part: 

 [1.72168821 1.74031106 1.92944251 1.67570641 1.32186894]
----------------------------------------------------------------------n
Weighted mean return in five part: 

 [0.20680867 0.19716919 0.18327156 0.24460084 0.31865894]
----------------------------------------------------------------------n
Difference between high part and low part: 

 -0.3998192664540337 0.11185027385453507
----------------------------------------------------------------------n
Alpha of mean return is 

 -0.0024482360347432212
----------------------------------------------------------------------n
Stat result of regression(mean): 

----------------------------------------------------------------------n
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic

## 2. Small Size

In [32]:
calresult(rankrt,mk,momentum, [20, 40, 60, 80], 87, rankrt.shape[1], 'Small')

Mean return in five part: 

 [2.34027119 2.19897885 2.35936903 2.26456681 1.73657908]
----------------------------------------------------------------------n
Weighted mean return in five part: 

 [0.389851   0.3792768  0.42474508 0.40447553 0.35448901]
----------------------------------------------------------------------n
Difference between high part and low part: 

 -0.6036921135589601 -0.035361994147722016
----------------------------------------------------------------------n
Alpha of mean return is 

 -0.005761002957488426
----------------------------------------------------------------------n
Stat result of regression(mean): 

----------------------------------------------------------------------n
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statisti

## 3. Big Size

In [34]:
calresult(rankrt,mk,momentum, [20, 40, 60, 80], 87, rankrt.shape[1], 'Big')

Mean return in five part: 

 [0.98700758 1.17509124 1.39537795 1.11352133 1.21530579]
----------------------------------------------------------------------n
Weighted mean return in five part: 

 [0.17252101 0.20882622 0.19480055 0.19006548 0.27056216]
----------------------------------------------------------------------n
Difference between high part and low part: 

 0.22829821298452993 0.09804114274290408
----------------------------------------------------------------------n
Alpha of mean return is 

 0.0028616792509390003
----------------------------------------------------------------------n
Stat result of regression(mean): 

----------------------------------------------------------------------n
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:

In [42]:
np.dot(np.array([3,2,1]).reshape(3,1), np.array([4,4,4,4]).reshape(1,4))

array([[12, 12, 12, 12],
       [ 8,  8,  8,  8],
       [ 4,  4,  4,  4]])

In [39]:
np.array([3,2,1]).reshape(1,3).shape

(1, 3)